# SuperBot Vertex LangGraph Walkthrough

This notebook demonstrates the SuperBot chatbot implemented with LangGraph and Google Vertex AI. You will load environment settings, inspect the reducer-backed state schema, compile the single-node graph, and compare invoke vs. streaming behaviors (`updates` vs. `values`).


## 1. Environment Setup
- Ensure Google Cloud Application Default Credentials (ADC) are configured (`gcloud auth application-default login`).
- Copy `configs/superbot.env.example` to `.env` (or supply your own env file) and fill in project, region, and model settings.
- Install dependencies from `requirements.txt` so `langgraph` and `langchain-google-vertexai` are available.


In [ ]:
from dotenv import load_dotenv

# Load environment variables (falls back to .env if no path supplied)
dotenv_loaded = load_dotenv()
print(f"dotenv loaded: {dotenv_loaded}")


## 2. Inspect the LangGraph State
The agent preserves conversation history in a reducer-managed list. The schema uses `typing.Annotated[..., add_messages]` so LangGraph appends new messages instead of replacing the state.


In [ ]:
from typing import get_type_hints

from super_bot_agent.state import State

# Display the reducer-backed type hints for the state schema
state_annotations = get_type_hints(State, include_extras=True)
state_annotations


## 3. Configure the Vertex Provider
`VertexConfig` validates environment variables and feeds them into `ChatVertexAI`. The helper surfaces descriptive errors when keys are missing or malformed.


In [ ]:
from super_bot_agent.provider import VertexConfig

try:
    vertex_config = VertexConfig.from_env()
    vertex_config
except Exception as exc:  # noqa: BLE001 - display errors during setup
    print(f"Configuration error: {exc}")


## 4. Compile the SuperBot Graph
With a valid configuration you can materialize the single-node LangGraph workflow. The helper returns a compiled graph ready for `invoke` or `stream` operations.


In [ ]:
from super_bot_agent import build_super_bot_graph, build_vertex_chat_model

try:
    llm = build_vertex_chat_model()
    graph = build_super_bot_graph(llm)
    graph
except Exception as exc:  # noqa: BLE001
    print(f"Graph initialization failed: {exc}")


### Visualize the Topology
For quick introspection LangGraph can render a text diagram of the compiled workflow.


In [ ]:
if "graph" in globals():
    try:
        graph.get_graph().print_ascii()
    except AttributeError as exc:
        print(f"Graph visualization unavailable in this LangGraph version: {exc}")
else:
    print("Graph is not initialized yet.")


## 5. Run an Invoke Turn
Invoke executes the graph synchronously and returns the accumulated state (human + assistant messages).


In [ ]:
from langchain_core.messages import HumanMessage


def render_messages(messages):
    for message in messages:
        content = message.content
        if isinstance(content, list):
            fragments = []
            for part in content:
                if isinstance(part, dict) and isinstance(part.get("text"), str):
                    fragments.append(part["text"])
                elif isinstance(part, str):
                    fragments.append(part)
            content = "".join(fragments)
        print(f"[{message.type}] {content}")


payload = {"messages": [HumanMessage(content="Hi SuperBot, what can you help me with?")]}

if "graph" in globals():
    invoke_result = graph.invoke(payload)
    render_messages(invoke_result["messages"])
else:
    print("Graph is not initialized yet.")


## 6. Streaming Modes
`graph.stream` exposes two educational perspectives:
- `updates`: incremental deltas emitted as soon as the node yields output.
- `values`: snapshots of the node state showing message accumulation at each stage.


In [ ]:
updates_payload = {"messages": [HumanMessage(content="Summarize the philosophy behind LangGraph reducers.")]}

if "graph" in globals():
    print("--- Streaming (updates) ---")
    for event in graph.stream(updates_payload, stream_mode="updates"):
        print(event)
else:
    print("Graph is not initialized yet.")


In [ ]:
values_payload = {"messages": [HumanMessage(content="Explain how SuperBot keeps track of prior conversational turns.")]}

if "graph" in globals():
    print("--- Streaming (values) ---")
    for snapshot in graph.stream(values_payload, stream_mode="values"):
        node_state = snapshot.get("super_bot", {})
        render_messages(node_state.get("messages", []))
        print("---")
else:
    print("Graph is not initialized yet.")


## 7. Next Steps
- Wrap this graph with LangServe or FastAPI to expose a web UI.
- Experiment with different Vertex models by editing `VERTEX_MODEL` in your environment.
- Extend the graph with additional nodes (planner, toolformer) once you are comfortable with the single-node reducer pattern.
